In [17]:
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import pandas as pd
import gensim
from gensim import corpora
from nltk.corpus import stopwords
import spacy


data = pd.read_csv("data.csv")
print(data['content'].info())
dataFiltered = data["content"][:35000]

stop_words = stopwords.words('english')

# function to remove stopwords
def remove_stopwords(text):
    textArr = text.split(' ')
    rem_text = " ".join([i for i in textArr if i not in stop_words])
    return rem_text

# remove stopwords from the text
dataFiltered=dataFiltered.apply(remove_stopwords)

# nlp = en_core_web_sm.load()
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']): 
       output = []
       for sent in texts:
             doc = nlp(sent) 
             output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags ])
       return output

text_list=dataFiltered.tolist()
#print(text_list[5])
tokenized_reviews = lemmatization(text_list)
#print(tokenized_reviews[5])

# -- Omar was here --------------------------------------------------------------------------------

dictionary = corpora.Dictionary(tokenized_reviews)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_reviews]

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(corpus=doc_term_matrix, id2word=dictionary, num_topics=15, random_state=100, chunksize=100, passes=10)
print("-"*10, "\n", "Topics\n", ldamodel.print_topics(num_words=10))



<class 'pandas.core.series.Series'>
RangeIndex: 50000 entries, 0 to 49999
Series name: content
Non-Null Count  Dtype 
--------------  ----- 
50000 non-null  object
dtypes: object(1)
memory usage: 390.8+ KB
None


c:\Python310\lib\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
c:\Python310\lib\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
c:\Python310\lib\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
c:\Python310\lib\site-packages\catalogue\__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
c:\Python310\lib\site-packages\catalogue\__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, doc_term_matrix, dictionary)
vis


C:\Users\eslam\AppData\Roaming\Python\Python310\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.011775  0.095162       1        1  23.094331
12     0.140826  0.105387       2        1   9.261808
6      0.098419  0.013420       3        1   7.713591
13     0.002643  0.209501       4        1   7.405444
14     0.067804  0.120429       5        1   7.105945
0      0.033303 -0.206191       6        1   6.008509
5     -0.079500 -0.190943       7        1   5.658831
1     -0.229716  0.001539       8        1   5.560353
4     -0.025035 -0.235748       9        1   4.615249
7      0.202992  0.083844      10        1   4.598519
11     0.098963 -0.061034      11        1   4.499494
2     -0.151248  0.013650      12        1   4.446996
9      0.192927 -0.115815      13        1   4.070986
3     -0.120663  0.115447      14        1   3.480177
10    -0.243490  0.051354      15        1   2.479768, topic_info=          Term          Freq         Total Category  logprob  loglift
696      woman  21480.000000  21480.000000  Default  30.0000  30.0000
503     police  14277.000000  14277.000000  Default  29.0000  29.0000
219   campaign  18431.000000  18431.000000  Default  28.0000  28.0000
1073      game   9924.000000   9924.000000  Default  27.0000  27.0000
5447   twitter  10456.000000  10456.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
338      first   1068.058811  13155.045665  Topic15  -4.6476   1.1860
163       year   1301.837968  37249.558324  Topic15  -4.4497   0.3431
79        last   1060.800296  19107.819348  Topic15  -4.6545   0.8059
152       time    970.013381  27783.658117  Topic15  -4.7439   0.3421
1497     night    850.567645   9142.219146  Topic15  -4.8753   1.3223

[1002 rows x 6 columns], token_table=       Topic      Freq Term
term                       
11024      5  0.999807    %
1879       4  0.062638    .
1879       5  0.744979    .
1879       7  0.042155    .
1879       8  0.035475    .
...      ...       ...  ...
704        8  0.019087   ’s
704       10  0.044094   ’s
704       12  0.014902   ’s
704       14  0.050218   ’s
704       15  0.027967   ’s

[2474 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 13, 7, 14, 15, 1, 6, 2, 5, 8, 12, 3, 10, 4, 11])

In [ ]:
print('\nPerplexity: ', ldamodel.log_perplexity(doc_term_matrix,total_docs=10000))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=ldamodel, texts=tokenized_reviews, dictionary=dictionary , coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.800897588314177

Coherence Score:  0.5083657165552928
